In [1]:
import numpy as np
import pandas as pd

import f_validate_g4s as ed

from datetime import timedelta
from time import time

pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_rows', 100)

In [95]:
# %load D:/BUDS_MID/__initdask__.py
"""
Created on Wed Feb 19 16:24:03 2020

@author: cmchico
"""
import dask.bag as db
from dask.distributed import Client, progress
from dask import delayed

client = Client(processes = True,n_workers=12)
client

Client Scheduler: tcp://127.0.0.1:59039 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 12 Cores: 12 Memory: 34.14 GB


In [88]:
path = 'D:/BUDS_MID/OJT ATM/ATM Forecasting Data New/'

In [3]:
test_dates = pd.date_range(start='01/01/2019',end='12/31/2019')

In [104]:
reload_df = pd.read_pickle(path+'Actual/80 Replenishment.pkl')
reload_df = reload_df.sort_values(['TID','DATE'],ignore_index=True)
reload_df['Pre-Emptive'] = reload_df['Pre-Emptive'].str.strip().fillna("NORMAL LOADING")
reload_df['date_str'] = reload_df.DATE.astype(str)


reload_df['Current Load Amount'] = reload_df['Current Load Amount'].astype(str).str.strip().str.replace("","")
select = reload_df['Current Load Amount']==""
reload_df.loc[select,'Current Load Amount'] = "0"
reload_df['Current Load Amount'] = reload_df['Current Load Amount'].astype(int)

reload_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212252 entries, 0 to 212251
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   Last Load Amount     212249 non-null  object        
 1   Current Load Amount  212252 non-null  int32         
 2   % Cash Return/Atm    212252 non-null  float64       
 3   Replen               212252 non-null  object        
 4   Pre-Emptive          212252 non-null  object        
 5   TID                  212252 non-null  object        
 6   DATE                 212252 non-null  datetime64[ns]
 7   date_str             212252 non-null  object        
dtypes: datetime64[ns](1), float64(1), int32(1), object(5)
memory usage: 12.1+ MB


In [105]:
reload_df['Pre-Emptive'].value_counts()

NORMAL LOADING           122370
PRE-EMPTIVE               67953
PRE-EMPTIVE DUE TO CF     14770
BPI INITIATIVE             7159
Name: Pre-Emptive, dtype: int64

In [106]:
actual=pd.read_pickle(path+'Actual/82 Aggregated TID WDL.pkl')
actual['AMOUNT_SUM'] = actual['WDL_AMT']
actual['date_str'] = actual.DATE.astype(str)
actual = actual[actual.DATE.isin(test_dates)].copy()
actual.TID = actual.TID.astype(str)
actual.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 299300 entries, 6440 to 2576304
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   DATE         299300 non-null  datetime64[ns]
 1   TID          299300 non-null  object        
 2   WDL_AMT      299300 non-null  float64       
 3   WDL_AMT_FIN  299300 non-null  float64       
 4   AMOUNT_SUM   299300 non-null  float64       
 5   date_str     299300 non-null  object        
dtypes: datetime64[ns](1), float64(3), object(2)
memory usage: 16.0+ MB


In [107]:
reload_df = reload_df[reload_df.DATE.isin(test_dates)].copy()
reload_df = reload_df.sort_values(['TID','DATE'],ignore_index=True)

In [108]:
first_reload = reload_df.drop_duplicates(subset=(['TID'])).copy()

In [109]:
# actual
atms_raw = actual.TID.unique().tolist()
atms = [atm for atm in atms_raw if atm in first_reload.TID.unique()]

print(f'No. of ATMs from Raw: {len(atms_raw)}')

print(f'NO of ATMs from G4S: {first_reload.TID.nunique()}')
print(f'NO of ATMs for simulation: {len(atms)}')

No. of ATMs from Raw: 820
NO of ATMs from G4S: 1560
NO of ATMs for simulation: 820


In [110]:
r = 0.05/365
reload_charge=2000
threshold=0.25

In [111]:
def par(atm):
    date_o = first_reload.loc[first_reload['TID'] == atm].reset_index().DATE[0]
    actual_atm = actual[(actual['TID'] == atm) & (actual.DATE >= date_o)].reset_index(drop=True).copy()
    
    g4s = reload_df[reload_df['TID'] == atm].set_index('date_str').copy()
    
    g4s_all_sched = ed.validate_g4s(actual_atm, g4s, r=r,reload_charge=reload_charge, threshold=threshold)
    
    g4s_all_sched.to_csv(path+'G4S Schedule/_sched_'+ atm+'.csv')
    ss=g4s_all_sched.groupby('cog_preemptive') \
                        [['reload_flag','cashout_flag','thresh_breach_flag','necessary_flag',
                                             'civ_cost','reload_cost']].sum().reset_index()
    ss['tid']=atm
    
    ss.to_csv(path+'G4S Schedule Perf/_schedperf_'+ atm+'.csv')
    return g4s_all_sched,ss

In [93]:
g4s_all_sched = dict()
g4s_all_ssum = dict()

print(f'No. of ATMs from Raw: {len(atms)}')

for atm in atms[296:300]:
    #first reload date
    print(atm)
    start = time()
    date_o = first_reload.loc[first_reload['TID'] == atm].reset_index().DATE[0]
    actual_atm = actual[(actual['TID'] == atm) & (actual.DATE >= date_o)].reset_index(drop=True).copy()
    
    g4s = reload_df[reload_df['TID'] == atm].set_index('date_str').copy()
    
    g4s_all_sched[atm] = ed.validate_g4s(actual_atm, g4s, r=r,reload_charge=reload_charge, threshold=threshold)
    
    g4s_all_sched[atm].to_csv(path+'G4S Schedule/_sched_'+ atm+'.csv')
    ss=g4s_all_sched[atm].groupby('cog_preemptive') \
                        [['reload_flag','cashout_flag','thresh_breach_flag','necessary_flag',
                                             'civ_cost','reload_cost']].sum().reset_index()
    ss['tid']=atm
    
    ss.to_csv(path+'G4S Schedule Perf/_schedperf_'+ atm+'.csv')
    g4s_all_ssum[atm] = ss
print(timedelta(seconds=time()-start))

No. of ATMs from Raw: 820
91018070
91018076
91018077
91018078
0:00:00.423520


In [115]:
dbatms = db.from_sequence(atms)
dbmaster = dbatms.map(par)

In [116]:
start=time()
df = dbmaster.compute()
print(timedelta(seconds=time()-start))

0:10:13.253472


In [122]:
g4s_perf = pd.concat([df[i][1] for i in range(len(atms))], ignore_index=True)

In [128]:
g4s_sched = pd.concat([df[i][0].assign(tid=atms[i]) for i in range(len(atms))], ignore_index=True)

In [130]:
g4s_perf.to_csv(path+'85 G4S perf 2019.csv')
g4s_sched.to_pickle(path+'85 G4S sched 2019.pkl')

In [131]:
g4s_sched

,date_str,AMOUNT_SUM,start_bal,start_reload,total_bal,end_bal,last_load_amt,threshold,reload_flag,cashout_flag,thresh_breach_flag,necessary_flag,civ_cost,reload_cost,total_cost,preemptive_flag,cog_preemptive,tid
0,2019-01-01,220300.00,0.00,5200000.00,5200000.00,4979700.00,5200000.00,1300000.00,1,0,0,0,682.15,2000,2682.15,1,NORMAL LOADING,91010018
1,2019-01-02,0.00,4979700.00,0.00,4979700.00,4979700.00,5200000.00,1300000.00,0,0,0,0,682.15,0,682.15,1,No Reload,91010018
2,2019-01-03,1631800.00,4979700.00,5200000.00,5200000.00,3568200.00,5200000.00,1300000.00,1,0,0,0,488.79,2000,2488.79,1,NORMAL LOADING,91010018
3,2019-01-04,2295300.00,3568200.00,0.00,3568200.00,1272900.00,5200000.00,1300000.00,0,0,1,0,174.37,0,174.37,1,No Reload,91010018
4,2019-01-05,2117100.00,1272900.00,0.00,1272900.00,-844200.00,5200000.00,1300000.00,0,1,1,0,0.00,0,0.00,0,No Reload,91010018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297806,2019-12-27,1634700.00,0.00,5200000.00,5200000.00,3565300.00,5200000.00,1300000.00,1,0,0,0,488.40,2000,2488.40,0,NORMAL LOADING,91016725
297807,2019-12-28,3281500.00,3565300.00,5200000.00,5200000.00,1918500.00,5200000.00,1300000.00,1,0,0,0,262.81,2000,2262.81,1,NORMAL LOADING,91016725
297808,2019-12-29,3161800.00,1918500.00,0.00,1918500.00,-1243300.00,5200000.00,1300000.00,0,1,1,0,0.00,0,0.00,1,No Reload,91016725
297809,2019-12-30,2005500.00,0.00,5200000.00,5200000.00,3194500.00,5200000.00,1300000.00,1,0,0,0,437.60,2000,2437.60,0,NORMAL LOADING,91016725
